In [ ]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
import pandas as pd
import numpy as np

import io
import os
import zipfile

import requests

In [ ]:
!pip install implicit

# load data

In [ ]:
content = requests.get("http://files.grouplens.org/datasets/movielens/ml-100k.zip").content

with zipfile.ZipFile(io.BytesIO(content)) as arc:
  train = arc.read("ml-100k/ua.base").decode().split("\n")
  test = arc.read("ml-100k/ua.test").decode().split("\n")
  movies = arc.read("ml-100k/u.item").decode(encoding='latin-1').split("\n")
  users = arc.read("ml-100k/u.user").decode(encoding='latin-1').split("\n")

In [ ]:
columns = ['user_id', 'movie_id', 'rating', 'timestamp']

data = [[x for x in e.split('\t')] for e in train]
train = pd.DataFrame(data, columns=columns).dropna().astype(int)

data = [[x for x in e.split('\t')] for e in test]
test = pd.DataFrame(data, columns=columns).dropna().astype(int)

columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

data = [[x for x in e.split('|')] for e in users]
users = pd.DataFrame(data, columns=columns).dropna().astype({'user_id': int, 'age': int})

genres = ['unknown', 'action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'noir',
          'horror', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western']
columns = ['movie_id', 'title', 'year', '-', 'url'] + genres
data = [[x for x in e.split('|')] for e in movies]
movies = pd.DataFrame(data, columns=columns).dropna().astype({'movie_id': int})
movies.drop(columns=['-', 'url'], inplace=True)
movies[genres] = movies[genres].astype(int)

# Utils

In [ ]:
def transform_predictions(k, user_ids, item_ids):
  return pd.DataFrame(
      data=np.c_[np.repeat(user_ids, k), item_ids.flatten(), [i + 1 for i in range(k)] * len(user_ids)],
      columns=['user_id', 'movie_id', 'rank']
  )


def get_full_df(k, user_ids, item_ids, true):
  preds = transform_predictions(k, user_ids, item_ids)
  preds = preds.merge(true, on=['user_id', 'movie_id'], how='outer')
  preds['rank'] = preds.groupby('user_id')['rank'].transform(lambda x: x.fillna(x.max() + 1))
  return preds

# Train model

In [ ]:
from implicit.cpu.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
pivot_table = train.pivot_table(index=['user_id'], columns=['movie_id'], values="rating").fillna(0)

als_model = AlternatingLeastSquares(factors=20, iterations=5, random_state=0)
als_model.fit(csr_matrix(pivot_table))

In [ ]:
test_users = test.user_id.unique()
len(test_users)

In [ ]:
ref_true = test[test.user_id.isin(test_users[:471])]
curr_true = test[test.user_id.isin(test_users[471:])]

In [ ]:
ids, scores = als_model.recommend(ref_true.user_id.unique() - 1, csr_matrix(pivot_table.loc[ref_true.user_id.unique()]), N=30, filter_already_liked_items=True)
ref = get_full_df(30, ref_true.user_id.unique(), ids, ref_true)

ids, scores = als_model.recommend(curr_true.user_id.unique() - 1, csr_matrix(pivot_table.loc[curr_true.user_id.unique()]), N=30, filter_already_liked_items=True)
curr = get_full_df(30, curr_true.user_id.unique(), ids, curr_true)

In [ ]:
from evidently.metrics import PrecisionTopKMetric, RecallTopKMetric, FBetaTopKMetric, MAPKMetric, NDCGKMetric
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

In [ ]:
report = Report(metrics=[
    PrecisionTopKMetric(k=5),
    RecallTopKMetric(k=5),
    FBetaTopKMetric(k=5),
    MAPKMetric(k=5),
    NDCGKMetric(k=5),
])
column_mapping=ColumnMapping(recommendations_type='rank', target='rating', prediction='rank')
report.run(reference_data=ref.fillna(0), current_data=curr.fillna(0), column_mapping=column_mapping)
report